In [1]:
import cv2
import numpy as np

def detect_fire_in_video(video_path, lower_fire=np.array([15, 100, 100]), upper_fire=np.array([35, 255, 255]), fire_threshold=0.01, min_fire_area=1000, display=True, save_output=False):
    """
    Detects fire in each frame of a video using HSV color thresholding and morphological operations.
    
    Parameters:
    - video_path: str, path to the video file
    - lower_fire: np.array, lower HSV threshold for fire-like colors
    - upper_fire: np.array, upper HSV threshold for fire-like colors
    - fire_threshold: float, percentage of fire pixels required to classify frame as containing fire
    - min_fire_area: int, minimum contour area to classify frame as containing fire
    - display: bool, whether to display the video with fire detection in real-time
    - save_output: bool, whether to save the output video with fire detection annotations
    
    Returns:
    - None (processes and optionally displays/saves video in real-time)
    """
    
    # Open the video file or camera feed
    cap = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print(f"Error: Unable to open video {video_path}")
        return
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Optional: Save output video with annotations
    if save_output:
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('output_fire_detection.avi', fourcc, fps, (width, height))
    
    while True:
        # Read each frame
        ret, frame = cap.read()
        
        if not ret:
            break  # End of video

        # Step 1: Convert the frame to HSV color space
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Step 2: Create a mask based on the color range
        fire_mask = cv2.inRange(hsv_frame, lower_fire, upper_fire)

        # Step 3: Perform morphological operations to clean the mask
        kernel = np.ones((5, 5), np.uint8)
        fire_mask_cleaned = cv2.morphologyEx(fire_mask, cv2.MORPH_OPEN, kernel)
        fire_mask_cleaned = cv2.morphologyEx(fire_mask_cleaned, cv2.MORPH_CLOSE, kernel)

        # Step 4: Classify fire based on pixel percentage and area
        fire_present = classify_fire_combined(fire_mask_cleaned, fire_threshold, min_fire_area)

        # Optional: Detect and draw contours around the fire regions
        if fire_present:
            contours, _ = cv2.findContours(fire_mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                if cv2.contourArea(contour) > min_fire_area:
                    cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)  # Draw green contours for fire regions

            # Add a label to indicate fire detection
            cv2.putText(frame, "Fire Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Optional: Display the frame with fire contours
        if display:
            cv2.imshow('Fire Detection', frame)

        # Save the output frame if required
        if save_output:
            out.write(frame)

        # Press 'q' to exit the video early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release video capture and writer objects
    cap.release()
    if save_output:
        out.release()
    cv2.destroyAllWindows()


def classify_fire_combined(fire_mask, fire_threshold=0.01, min_fire_area=1000):
    """
    Classify whether fire is present using both pixel percentage and fire region area.
    
    Parameters:
    - fire_mask: np.array, binary mask where fire regions are white (255)
    - fire_threshold: float, the percentage threshold for pixel-based fire detection
    - min_fire_area: int, the minimum total area of fire regions for area-based fire detection
    
    Returns:
    - bool: True if fire is detected by either method, False otherwise
    """
    
    # Fire pixel percentage check
    fire_pixel_count = np.count_nonzero(fire_mask)
    total_pixel_count = fire_mask.size
    fire_percentage = fire_pixel_count / total_pixel_count
    
    # Fire area check
    contours, _ = cv2.findContours(fire_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    total_fire_area = sum(cv2.contourArea(contour) for contour in contours)
    
    # Fire detected if either condition is true
    if fire_percentage >= fire_threshold or total_fire_area >= min_fire_area:
        return True
    else:
        return False


In [3]:
detect_fire_in_video('test_videos/test2.mp4', display=True, save_output=True)